## Checking TRUSTED TASK with the "source" data from raw csv.

### Imports | Functions and variables

In [1]:
import pandas as pd

In [17]:
def unpivot_via_pandas(df: pd.DataFrame, name_column: str) -> pd.DataFrame:
    """
    Performs an unpivot operation on a DataFrame using Pandas.

    Args:
        df (pd.DataFrame): The DataFrame containing the CSV data.
        name_column (str): The name for the unpivoted value column.

    Returns:
        pd.DataFrame: The resulting DataFrame after the unpivot operation.
    """
    # Lista de colunas que não são datas
    non_date_columns = ['Province/State', 'Country/Region', 'Lat', 'Long']

    # Realiza o unpivot usando melt
    unpivoted_df = df.melt(id_vars=non_date_columns, var_name='Date', value_name='Value')

    # Converte a coluna de data para o formato datetime
    unpivoted_df['Date'] = pd.to_datetime(unpivoted_df['Date'], format='%m/%d/%y')

    # Adiciona colunas para mês e ano
    unpivoted_df['mes'] = unpivoted_df['Date'].dt.month
    unpivoted_df['ano'] = unpivoted_df['Date'].dt.year

    # Renomeia as colunas conforme necessário
    unpivoted_df = unpivoted_df.rename(columns={
        'Country/Region': 'pais',
        'Province/State': 'estado',
        'Lat': 'latitude',
        'Long': 'longitude',
        'Date': 'data',
        'Value': name_column
    })

    return unpivoted_df


In [18]:
csv_confirmed_file_path = '/home/gabrielphilot/cases/radix/desafio-data-engineer/datalake/raw/covid19/time_series_covid19_confirmed_global.csv'
csv_deaths_file_path = '/home/gabrielphilot/cases/radix/desafio-data-engineer/datalake/raw/covid19/time_series_covid19_deaths_global.csv'
csv_recovered_file_path = '/home/gabrielphilot/cases/radix/desafio-data-engineer/datalake/raw/covid19/time_series_covid19_recovered_global.csv'


In [19]:
df_confirmed = pd.read_csv(csv_confirmed_file_path)
df_deaths = pd.read_csv(csv_deaths_file_path)
df_recovered = pd.read_csv(csv_recovered_file_path)


unpivoted_confirmed = unpivot_via_pandas(df_confirmed, 'confirmed')
unpivoted_deaths = unpivot_via_pandas(df_deaths, 'deaths')
unpivoted_recovered = unpivot_via_pandas(df_recovered, 'recovered')

### Here my goal was to only see if some random country of each csv,
### has the same value in the source and in the trusted task after the join in the airflow.

In [38]:
grouped_df = unpivoted_confirmed.groupby(['pais', 'ano', 'mes'])['confirmed'].sum().reset_index()
australia_months_by_year = grouped_df[grouped_df['pais'] == 'Australia'].drop(columns=['pais'])
australia_months_by_year

,ano,mes,confirmed
136,2020,1,38
137,2020,2,431
138,2020,3,35679
139,2020,4,188206
140,2020,5,217625
141,2020,6,222611
142,2020,7,361720
143,2020,8,711091
144,2020,9,800899
145,2020,10,848364


In [35]:
grouped_df_death = unpivoted_deaths.groupby(['pais', 'ano', 'mes'])['deaths'].sum().reset_index()
brazil_deaths = grouped_df_death[grouped_df_death['pais'] == 'Brazil'].drop(columns=['pais'])

brazil_deaths

,ano,mes,deaths
391,2020,1,0
392,2020,2,0
393,2020,3,976
394,2020,4,66571
395,2020,5,509030
396,2020,6,1353392
397,2020,7,2370476
398,2020,8,3345405
399,2020,9,4004352
400,2020,10,4732789


In [34]:
# Same value as excel  (value in the raw csv value without unPivot).

grouped_df_death_total = grouped_df_death['deaths'].sum()
grouped_df_death_total

65053022

In [36]:
grouped_df_recover = unpivoted_recovered.groupby(['pais', 'ano', 'mes'])['recovered'].sum().reset_index()
Egypt_recover = grouped_df_recover[grouped_df_recover['pais'] == 'Egypt'].drop(columns=['pais'])

Egypt_recover 

,ano,mes,recovered
901,2020,1,0
902,2020,2,2
903,2020,3,1407
904,2020,4,20364
905,2020,5,103095
906,2020,6,374233
907,2020,7,849040
908,2020,8,1825324
909,2020,9,2574263
910,2020,10,3043034
